In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '../result_fc/'

In [3]:
gds_list_0 = pd.read_csv(path+'gds_list_0.csv')
gds_list_1 = pd.read_csv(path+'gds_list_1.csv')
gdsclss_list_0 = pd.read_csv(path+'gdsclss_list_0.csv')
gdsclss_list_1 = pd.read_csv(path+'gdsclss_list_1.csv')
str_list = pd.read_csv(path+'str_list.csv')
tr_list_0_convert = pd.read_csv(path+'tr_list_0_convert.csv')
tr_list_1_convert = pd.read_csv(path+'tr_list_1_convert.csv')
tr_dtl_list_0 = pd.read_csv(path+'tr_dtl_list_0.csv')
tr_dtl_list_1 = pd.read_csv(path+'tr_dtl_list_1.csv')


## 0 - A,B마트 1 - C,D마트

## 1. gds_list_0, gds_list_1: 제품 목록
- CO_CD: 회사 코드
- STR_CD: 매장 코드
- PLU_CD: 제품 코드로 880으로 시작하는 바코드도 있지만 매장에서 직접 생성하여 등록한 코드가 혼재
- GDS_NM: 제품 명
- GDS_CLSS_CD: 제품 카테고리 코드(자세한 코드 구조는 제품 카테고리 데이터 참조)
- GDS_TP_NM: 제품 구분으로 [일반/저울상품/사이드상품/원타임/공병/배송료]로 구분
- VEN_CD: 제품 공급사 코드로 매장에서 관리하는 번호기 때문에 다른 매장 간 같은 코드로 중복되더라도 다른 제품 공급사일 수 있음

### gds_list_0

In [7]:
gds_list_0.head()

,CO_CD,STR_CD,PLU_CD,GDS_NM,GDS_CLSS_CD,GDS_TP_NM,VEN_CD
0,1001294,12018,1,배송비,999999,배송료,0
1,1001294,12018,2,사은행사쿠폰,999999,배송료,0
2,1001294,12018,91770,코지유리물병,93331,일반,0
3,1001294,12018,91770,크지물병,91701,일반,0
4,1001294,12018,50390427,POM석류,12579,일반,0


In [8]:
gds_list_0['PLU_CD'].value_counts()

41820070163      5
51000129116      5
33200000228      5
47400241442      5
79426261609      5
                ..
8801157550408    1
8801157550538    1
8801157550545    1
8801157550552    1
990942           1
Name: PLU_CD, Length: 84945, dtype: int64

In [9]:
gds_list_0[['GDS_CLSS_CD', 'GDS_TP_NM']].value_counts()

GDS_CLSS_CD  GDS_TP_NM
91701        일반           7477
90301        일반           5469
140101       일반           3248
10101        일반           2912
11502        일반           2910
                          ... 
89999        저울상품            1
95202        일반              1
10505        저울상품            1
11060        저울상품            1
40101        저울상품            1
Length: 612, dtype: int64

In [10]:
gds_list_0_a = gds_list_0[gds_list_0['STR_CD']==12018]

In [11]:
gds_list_0_b = gds_list_0[gds_list_0['STR_CD']==72005]

### gds_list_1

In [12]:
gds_list_1

,CO_CD,STR_CD,PLU_CD,GDS_NM,GDS_CLSS_CD,GDS_TP_NM,VEN_CD
0,1001993,142011,0,종랑제75ㄴ,999999,일반,0
1,1001993,142011,1,배송비,999999,배송료,0
2,1001993,142011,2,사은행사쿠폰,999999,배송료,0
3,1001993,142011,12003,얼음물,999999,일반,0
4,1001993,142011,22002,써니텐=사이다,999999,일반,0
...,...,...,...,...,...,...,...
70057,1002056,168009,9805817760205,포카칩,898989,일반,46
70058,1002056,168009,9808205106111,매입에서등록한상품,997979,일반,30
70059,1002056,168009,9808862878406,월드콘,898989,일반,9
70060,1002056,168009,9880913160015,매입에서등록한상품,997979,일반,34


In [13]:
gds_list_1['GDS_TP_NM'].value_counts()

일반      69154
저울상품      904
배송료         4
Name: GDS_TP_NM, dtype: int64

### 2. gdsclss_list_0, gdsclss_list_1: 제품 카테고리 목록
- CO_CD: 회사 코드
- STR_CD: B/C/D마트는 000000으로 표기되고 A마트만 A마트의 매장코드가 표시됨(A마트는 다점포회사 소속으로 매장마다 카테고리 구분하여 관리)
- GDS_CLSS_CD: 제품 카테고리 코드로 2자리, 4자리, 6자리로 구성, 6자리 코드의 앞 4자리와 4자리 코드의 앞 2자리는 각 상위 카테고리의 코드
- GDS_CLSS_NM: 제품 카테고리 명
- GDS_CLSS_LVL_TP: 카테고리 Level로 1/2/3으로 표시
- UP_GDS_CLSS_CD: 상위 카테고리 코드
- GDS_CLSS_GRP_TP: 카테고리 구분으로 1은 공산품, 2는 식자재, 3은 신선식품(실수로 연결을 못해서 코드 그대로 표시)

### gdsclss_list_0

In [14]:
gdsclss_list_0.head()

,CO_CD,STR_CD,GDS_CLSS_CD,GDS_CLSS_NM,GDS_CLSS_LVL_TP,UP_GDS_CLSS_CD,GDS_CLSS_GRP_TP
0,1001294,12018,1,식품,1,0,1
1,1001294,12018,101,가공농산물,2,1,1
2,1001294,12018,10151,가공농산물,3,101,1
3,1001294,12018,10152,김치류,3,101,1
4,1001294,12018,10153,소스,3,101,1


In [15]:
gdsclss_list_0['GDS_CLSS_GRP_TP'].value_counts()

1    772
Name: GDS_CLSS_GRP_TP, dtype: int64

In [16]:
gdsclss_list_0[['GDS_CLSS_CD', 'GDS_CLSS_NM']].value_counts()

GDS_CLSS_CD  GDS_CLSS_NM
1            식품             1
20199        기타세면류          1
20504        물티슈            1
20599        기타애완용품         1
20601        생리대            1
                           ..
10399        기타가공식품류        1
10401        아이스크림          1
             햄,소세지류         1
10402        가공어묵류          1
999999       미등록            1
Length: 772, dtype: int64

In [17]:
gdsclss_list_0[['STR_CD', 'CO_CD']].value_counts()

STR_CD  CO_CD  
12018   1001294    607
0       1001739    165
dtype: int64

### gdsclss_list_1

In [18]:
gdsclss_list_1

,CO_CD,STR_CD,GDS_CLSS_CD,GDS_CLSS_NM,GDS_CLSS_LVL_TP,UP_GDS_CLSS_CD,GDS_CLSS_GRP_TP
0,1001993,0,1,식 품 류,1,0,1
1,1001993,0,101,가공식품,2,1,1
2,1001993,0,10101,가공식품,3,101,1
3,1001993,0,10199,기타-가공식품,3,101,1
4,1001993,0,102,식 용 유,2,1,1
...,...,...,...,...,...,...,...
362,1002056,0,99,미등록,1,0,1
363,1002056,0,9979,임시핸디매입등록용,2,99,1
364,1002056,0,997979,임시핸디매입등록용,3,9979,1
365,1002056,0,9999,미등록,2,99,1


In [19]:
gdsclss_list_1['GDS_CLSS_NM'].value_counts()

액세서리       5
위생용품       4
아이스크림      4
야    채     3
종량봉투       3
          ..
기타-공산면세    1
비식품면세      1
공산면세       1
기타-식품면세    1
식 품 류      1
Name: GDS_CLSS_NM, Length: 225, dtype: int64

## 3. str_list: 매장 목록
- CO_CD: 회사 코드
- STR_CD: 매장 코드
- STR_NM: (마스킹 된) 매장 명
- CM_ITM_NM: 매장 크기 구분, 30평/50평/100평/300평/500평/1000평 총 6종류
- POS_QTY: 포스 대수(토마토 솔루션 가입 당시)
- OPN_DT: 매장 오픈 일자

In [20]:
str_list

,CO_CD,STR_CD,STR_NM,CM_ITM_NM,POS_QTY,OPN_DT
0,1001294,12018,A마트,100평대,9,20201220
1,1001739,72005,B마트,100평대,3,20210526
2,1001993,142011,C마트,30평대,1,20210805
3,1002056,168009,D마트,100평대,2,20210831


## 4. tr_list_0, tr_list_1: 매출 데이터 마스터(장바구니 단위)
- SAL_DT: 매출 등록 일자
- STR_CD: 매장 코드
- POS_NO: 포스 번호
- TRAN_NO: 포스 거래 순번 -- 위의 4개 항목을 합쳐 하나의 장바구니
- SALE_KIND_TP_NM: 매출 구분으로 매출/전체취소/배달입금/배달착불만 대상으로 처리하면 됩니다. 나머지는 기능적이거나 오프라인 외 매출 등 입니다.
- SALE_RTN_SGN: 금액에 대한 부호로 1과 -1로 구분, -1은 반품
- REAL_SAL_DT: 실제 판매 일자
- REAL_SAL_TTM: 실제 판매 시각
- PAY_AMT: 지불 금액
- MSHP_ID: (마스킹 된) 회원 번호
- ORN_SAL_DT: 원 매출 등록 일자(반품 시 반품 대상이 되는 매출을 특정하기 위함)
- ORN_POS_NO: 원 매출 포스 번호(반품 시 반품 대상이 되는 매출을 특정하기 위함)
- ORN_TRAN_NO: 원 매출 포스 거래 순번(반품 시 반품 대상이 되는 매출을 특정하기 위함)

### tr_list_0

In [64]:
tr_list_0_convert.head()

,SAL_DT,STR_CD,POS_NO,TRAN_NO,SALE_KIND_TP_NM,SALE_RTN_SGN,REAL_SAL_DT,REAL_SAL_TTM,PAY_AMT,MSHP_ID,ORN_SAL_DT,ORN_POS_NO,ORN_TRAN_NO
0,20210901,12018,01,1,영업개시,1,20210901,65745,0.000000000,NaN,NaN,NaN,NaN
1,20210901,12018,01,2,준비금,1,20210901,65758,0.000000000,NaN,NaN,NaN,NaN
2,20210901,12018,01,3,매출,1,20210901,71138,9000000.000000000,NaN,NaN,NaN,NaN
3,20210901,12018,01,4,전체취소,1,20210901,85353,0.000000000,NaN,NaN,NaN,NaN
4,20210901,12018,01,5,매출,1,20210901,91245,3650.000000000,012018_0,NaN,NaN,NaN


In [70]:
tr_list_0_convert.MSHP_ID.value_counts()

012018_398      793
012018_135      718
012018_945      715
012018_2207     682
012018_52       669
               ... 
012018_18188      1
012018_18193      1
012018_18194      1
012018_18200      1
072005_6103       1
Name: MSHP_ID, Length: 29253, dtype: int64

### tr_list_1

In [23]:
tr_list_1_convert.head()

,SAL_DT,STR_CD,POS_NO,TRAN_NO,SALE_KIND_TP_NM,SALE_RTN_SGN,REAL_SAL_DT,REAL_SAL_TTM,PAY_AMT,MSHP_ID,ORN_SAL_DT,ORN_POS_NO,ORN_TRAN_NO
0,20210901,142011,1,1,영업개시,1,20210901,65138,0.0,NaN,NaN,NaN,NaN
1,20210901,142011,1,2,매출,1,20210901,70033,4000.0,NaN,NaN,NaN,NaN
2,20210901,142011,1,3,매출,1,20210901,72058,5700.0,NaN,NaN,NaN,NaN
3,20210901,142011,1,4,매출,1,20210901,73457,3700.0,NaN,NaN,NaN,NaN
4,20210901,142011,1,5,매출,1,20210901,73903,28100.0,NaN,NaN,NaN,NaN


In [24]:
tr_list_1_convert['POS_NO'].value_counts()

1    155008
2       867
Name: POS_NO, dtype: int64

## 5. tr_dtl_list_0, tr_dtl_list_1: 매출 데이터 상세(매출 제품)
- SAL_DT: 매출 등록 일자
- STR_CD: 매장 코드
- POS_NO: 포스 번호
- TRAN_NO: 포스 거래 순번 -- 위의 4개 항목을 합쳐 하나의 장바구니
- SEQ_NO: 매출 별 상품 순번
- DESI_CNCL_TP_NM: 취소 구분으로 정상판매/지정취소/전체취소 총 3종류이며 지정취소와 전체취소는 거래 후 환불이 아닌 바코드 찍을 당시 삭제한 것을 기록한 것으로 정상판매만 사용해도 무방
- SALE_RTN_SGN: 금액에 대한 부호로 1과 -1로 구분, -1은 반품
- PLU_CD: 제품 코드
- GDS_CLSS_CD: 제품 카테고리 코드로 (예상하기로는) 거래 당시 등록되어있던 카테고리를 기록하는 목적인 것으로 판단(무시해도 좋을 것 같습니다.)
- SALE_QTY: 수량
- SALE_PRC: 단가
- SALE_AMT: 총 금액(수량X단가) - 전체 할인 금액
- SGDS_DC_TP_NM: 단품 할인 구분이라는 항목으로 정상/단품할인/고객지정가할인/할인특매 총 4개로 구분되나 매장마다 사용하는 용도가 다른 것으로 보입니다.
- SGDS_DC_PRC: 단품 할인 단가
- SUBTOT_DC_AMT: 할인 소계(제품에 적용된 할인 총액)
- EVT_DC_AMT: 행사 할인 금액
- CPON_DC_AMT: 쿠폰 할인 금액
- SCALES_GDS_TP_NM: 저울 상품 구분, 일반/일반저울/중량저울, 저울상품이란 저울로 측정된 무게로 가격을 책정하는 경우로 주로 신선(야채, 육류) 등을 구분하는데 사용됨
- GDS_CPON_DC_AMT: 상품 쿠폰 할인 금액
- BASKET_CPON_DC_AMT: 장바구니 쿠폰 할인 금액

### tr_dtl_list_0

In [25]:
tr_dtl_list_0.head(30)

,SAL_DT,STR_CD,POS_NO,TRAN_NO,SEQ_NO,DESI_CNCL_TP_NM,SALE_RTN_SGN,PLU_CD,GDS_CLSS_CD,SALE_QTY,SALE_PRC,SALE_AMT,SGDS_DC_TP_NM,SGDS_DC_PRC,SUBTOT_DC_AMT,EVT_DC_AMT,CPON_DC_AMT,SCALES_GDS_TP_NM,GDS_CPON_DC_AMT,BASKET_CPON_DC_AMT
0,20210901,12018,01,3,1,정상판매,1,8801039939499,12301.0,180.0,3000.0,540000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
1,20210901,12018,01,3,2,정상판매,1,8801045312316,12101.0,360.0,1000.0,360000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
2,20210901,12018,01,3,3,정상판매,1,8801045291314,12101.0,360.0,1000.0,360000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
3,20210901,12018,01,3,4,정상판매,1,8803712127018,11363.0,180.0,2700.0,486000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
4,20210901,12018,01,3,5,정상판매,1,8801045890333,12102.0,180.0,9500.0,1710000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
5,20210901,12018,01,3,6,정상판매,1,8801007745671,12104.0,180.0,3600.0,648000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
6,20210901,12018,01,3,7,정상판매,1,8801007745718,12104.0,180.0,3600.0,648000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
7,20210901,12018,01,3,8,정상판매,1,8801043015264,11060.0,180.0,3500.0,630000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
8,20210901,12018,01,3,9,정상판매,1,8801045522234,11060.0,180.0,2500.0,450000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
9,20210901,12018,01,3,10,정상판매,1,8809398060045,10151.0,360.0,1800.0,648000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0


### tr_dtl_list_1

In [26]:
tr_dtl_list_1.head()

,SAL_DT,STR_CD,POS_NO,TRAN_NO,SEQ_NO,DESI_CNCL_TP_NM,SALE_RTN_SGN,PLU_CD,GDS_CLSS_CD,SALE_QTY,SALE_PRC,SALE_AMT,SGDS_DC_TP_NM,SGDS_DC_PRC,SUBTOT_DC_AMT,EVT_DC_AMT,CPON_DC_AMT,SCALES_GDS_TP_NM,GDS_CPON_DC_AMT,BASKET_CPON_DC_AMT
0,20210901,142011,1,2,1,정상판매,1,8801097250048,999999.0,1.0,1500.0,1500.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0
1,20210901,142011,1,2,2,정상판매,1,8801052171043,10301.0,1.0,1300.0,1300.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0
2,20210901,142011,1,2,3,정상판매,1,8801007028705,10101.0,1.0,1200.0,1200.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0
3,20210901,142011,1,3,1,정상판매,1,205105,110101.0,1.0,1300.0,1300.0,정상,0.0,0.0,0.0,0.0,일반저울,0.0,0.0
4,20210901,142011,1,3,2,정상판매,1,8801007112480,999999.0,1.0,1200.0,1200.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0


## EDA

In [9]:
tr_list_0_convert.MSHP_ID.notna().mean()

0.7327838100820129

In [77]:
tr_list_0 = tr_list_0_convert.copy()

In [80]:
tr_list_0 = tr_list_0[tr_list_0['MSHP_ID'].notna()]

In [83]:
tr_list_0[tr_list_0['MSHP_ID']=='012018_0']

,SAL_DT,STR_CD,POS_NO,TRAN_NO,SALE_KIND_TP_NM,SALE_RTN_SGN,REAL_SAL_DT,REAL_SAL_TTM,PAY_AMT,MSHP_ID,ORN_SAL_DT,ORN_POS_NO,ORN_TRAN_NO
4,20210901,12018,01,5,매출,1,20210901,91245,3650.0,012018_0,NaN,NaN,NaN
13558,20210905,12018,01,8,매출,1,20210905,70256,18900.0,012018_0,NaN,NaN,NaN
20983,20210907,12018,03,9,매출,1,20210907,73841,4980.0,012018_0,NaN,NaN,NaN
21065,20210907,12018,03,91,매출,1,20210907,101028,32840.0,012018_0,NaN,NaN,NaN
21682,20210907,12018,04,168,매출,1,20210907,130802,38440.0,012018_0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023800,20220808,12018,03,8,매출,1,20220808,90731,6480.0,012018_0,NaN,NaN,NaN
1025356,20220809,12018,01,8,매출,1,20220809,70402,4200.0,012018_0,NaN,NaN,NaN
1034048,20220812,12018,01,16,매출,1,20220812,71656,9800.0,012018_0,NaN,NaN,NaN
1048699,20220817,12018,01,12,매출,1,20220817,70238,16620.0,012018_0,NaN,NaN,NaN


In [62]:
tr_list_0.MSHP_ID.value_counts()

012018_398      793
012018_135      718
012018_945      715
012018_2207     682
012018_52       669
               ... 
012018_18188      1
012018_18193      1
012018_18194      1
012018_18200      1
072005_6103       1
Name: MSHP_ID, Length: 29253, dtype: int64

In [5]:
tr_dtl_list_0

,SAL_DT,STR_CD,POS_NO,TRAN_NO,SEQ_NO,DESI_CNCL_TP_NM,SALE_RTN_SGN,PLU_CD,GDS_CLSS_CD,SALE_QTY,SALE_PRC,SALE_AMT,SGDS_DC_TP_NM,SGDS_DC_PRC,SUBTOT_DC_AMT,EVT_DC_AMT,CPON_DC_AMT,SCALES_GDS_TP_NM,GDS_CPON_DC_AMT,BASKET_CPON_DC_AMT
0,20210901,12018,01,3,1,정상판매,1,8801039939499,12301.0,180.0,3000.0,540000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
1,20210901,12018,01,3,2,정상판매,1,8801045312316,12101.0,360.0,1000.0,360000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
2,20210901,12018,01,3,3,정상판매,1,8801045291314,12101.0,360.0,1000.0,360000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
3,20210901,12018,01,3,4,정상판매,1,8803712127018,11363.0,180.0,2700.0,486000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
4,20210901,12018,01,3,5,정상판매,1,8801045890333,12102.0,180.0,9500.0,1710000.0,할인특매,0.0,0.0,0.0,0.0,일반,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149885,20220901,72005,03,333,2,정상판매,1,22000422,110101.0,1.0,490.0,490.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0
6149886,20220901,72005,03,333,3,정상판매,1,8801094012403,10203.0,1.0,3200.0,3200.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0
6149887,20220901,72005,03,333,4,정상판매,1,8801019605505,10101.0,1.0,2500.0,2500.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0
6149888,20220901,72005,03,333,5,정상판매,1,8801117694807,10101.0,1.0,2100.0,2100.0,정상,0.0,0.0,0.0,0.0,일반,0.0,0.0


In [45]:
tr_dtl_list_0[tr_dtl_list_0['']]